In [ ]:
# ================================================================================
# COMPLETE PORTFOLIO ANALYSIS PIPELINE - CLEAN VERSION
# ================================================================================
# Organized pipeline for EP and Stats portfolio analysis

# IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import io
import tempfile

try:
    from openpyxl import load_workbook
    from openpyxl.drawing.image import Image
    EXCEL_PLOTS_AVAILABLE = True
    print("Excel image support available")
except ImportError:
    EXCEL_PLOTS_AVAILABLE = False
    print("Warning: openpyxl not available. Install with: pip install openpyxl")

print("All imports loaded successfully")


### Pipeline
From Analysis Output (from hdlosses) -> Excel sheets with plots

In [ ]:
#starting off with transferring the files to the local/appropriate path
#doing this from the remote desktop
#Moving files from HD losses to a preferred directory
import os
import shutil
import pandas as pd

source_dir = r"\\ca1fs01\ME_RESULTS\losses-hd"
destination_base = r"D:\AshleyY\GR_DK_NO_NL\NO_RESULTS"

portfolios = {
    "Expected":[207963, 207965, 207968, 207971, 207974, 207976]}

for portfolio, folders in portfolios.items():
    dest_folder = os.path.join(destination_base, portfolio)
    os.makedirs(dest_folder, exist_ok=True)

    for folder_id in folders:
        folder_name = str(folder_id)
        src_path = os.path.join(source_dir, folder_name)
        dst_path = os.path.join(dest_folder, folder_name)

        if os.path.exists(src_path) and os.path.isdir(src_path):
            if not os.path.exists(dst_path):
                shutil.copytree(src_path, dst_path)
                print(f"Copied folder: {folder_name} -> Portfolio {portfolio}")
            else:
                print(f"Skipped: {folder_name} already exists in Portfolio {portfolio}")
        else:
            print(f"Missing: Folder {folder_name} not found in source directory.")

In [ ]:
def ep_port_aggregating_files(results_base_path, loss_type):
    """
    Aggregates EP portfolio files from Results directory.
    Expected folder and sample folders are all at the same level in Results directory.
    
    Args:
        results_base_path: Path to Results directory containing all folders
        loss_type: "GU" or "GR"
    """
    import pandas as pd
    import os
    import re
    
    print(f"Processing EP Portfolio data for Loss Type: {loss_type}")
    print(f"Results Directory: {results_base_path}")
    
    df = pd.DataFrame()
    sample_sizes = []
    
    # Validate loss_type
    if loss_type not in ['GU', 'GR']:
        print(f"Warning: Unexpected loss_type '{loss_type}'. Expected 'GU' or 'GR'")
        return pd.DataFrame()
    
    if not os.path.exists(results_base_path):
        print(f"Results directory not found: {results_base_path}")
        return pd.DataFrame()
    
    result_folders = [d for d in os.listdir(results_base_path) 
                     if os.path.isdir(os.path.join(results_base_path, d))]
    
    print(f"Found {len(result_folders)} folders")
    
    # Step 1: Find and load the Expected folder
    expected_found = False
    for folder_name in result_folders:
        if "Expected" in folder_name:
            folder_path = os.path.join(results_base_path, folder_name)
            expected_file_path = os.path.join(folder_path, "EP", "Portfolio", loss_type, "EP100.parquet")
            
            if os.path.exists(expected_file_path):
                df = pd.read_parquet(expected_file_path)
                df['Expected'] = df['Loss']
                df['LossType'] = loss_type
                expected_found = True
                print(f"Loaded Expected data from folder: {folder_name}")
                break
    
    if not expected_found:
        print(f"No Expected folder found for {loss_type}")
        return pd.DataFrame()
    
    # Step 2: Process all sample folders
    processed_files = 0
    for folder_name in result_folders:
        if "Expected" not in folder_name:  # Skip the Expected folder
            folder_path = os.path.join(results_base_path, folder_name)
            sample_file_path = os.path.join(folder_path, "EP", "Portfolio", loss_type, "EP100.parquet")
            
            if os.path.exists(sample_file_path):
                try:
                    df_temp = pd.read_parquet(sample_file_path)
                    
                    # Extract sample size from folder name
                    sample_size = None
                    match = re.search(r'_S(\d+)_', folder_name)
                    if match:
                        sample_size = int(match.group(1))
                    else:
                        # Fallback: use processed count + 1
                        sample_size = processed_files + 1
                    
                    # Add sample column
                    column_name = f'S{sample_size}'
                    df[column_name] = df_temp['Loss']
                    sample_sizes.append(sample_size)
                    processed_files += 1
                    print(f"Added {loss_type} sample from {folder_name}: {column_name}")
                    
                except Exception as e:
                    print(f"Error processing {folder_name}: {str(e)}")
                    continue
    
    sample_sizes.sort()
    print(f"Processed {processed_files} sample folders")
    print(f"Sample sizes: {sample_sizes}")
    
    # Calculate differences vs S32
    for sample_size in sample_sizes:
        s_col = f'S{sample_size}'
        if sample_size != 32 and s_col in df.columns and 'S32' in df.columns:
            df[f'{s_col}VSS32'] = (df[s_col] - df['S32']) / df['S32'] * 100

    # Calculate S32 vs Expected (this is the key comparison)
    if 'S32' in df.columns and 'Expected' in df.columns:
        df['S32vsExpected'] = (df['S32'] - df['Expected']) / df['Expected'] * 100

    # Reorder columns
    base_cols = [col for col in df.columns if not col.startswith('S') or col == 'S32vsExpected']

    s_cols = sorted(
        [col for col in df.columns if col.startswith('S') and 'vs' not in col and col[1:].isdigit()],
        key=lambda x: int(x[1:])
    )

    vs_cols = sorted(
        [col for col in df.columns if 'VSS32' in col and col[1:col.find('V')].isdigit()],
        key=lambda x: int(x[1:x.find('V')])
    )

    if 'S32vsExpected' in base_cols:
        base_cols.remove('S32vsExpected')

    new_order = base_cols + s_cols + vs_cols + ['S32vsExpected']
    df = df[new_order]

    # Filter to only OEP data
    if 'EPType' in df.columns:
        df = df[df['EPType'] == 'OEP']

    return df


In [ ]:
ep_port_aggregating_files(r'C:\Users\Ashley.Yang\Downloads\DK_RES_GR_GU\Results', "GR")

In [ ]:
# Stats Portfolio Aggregation Function - Corrected Structure
def stat_port_aggregating_files(results_base_path, loss_type):
    """
    Aggregates Stats portfolio files from Results directory.
    Expected folder and sample folders are all at the same level in Results directory.
    Uses AAL (Average Annual Loss) instead of Loss.
    
    Args:
        results_base_path: Path to Results directory containing all folders
        loss_type: "GU" or "GR"
    """
    import pandas as pd
    import os
    import re
    
    print(f"Processing Stats Portfolio data for Loss Type: {loss_type}")
    print(f"Results Directory: {results_base_path}")
    
    df = pd.DataFrame()
    sample_sizes = []
    
    # Validate loss_type
    if loss_type not in ['GU', 'GR']:
        print(f"Warning: Unexpected loss_type '{loss_type}'. Expected 'GU' or 'GR'")
        return pd.DataFrame()
    
    if not os.path.exists(results_base_path):
        print(f"Results directory not found: {results_base_path}")
        return pd.DataFrame()
    
    result_folders = [d for d in os.listdir(results_base_path) 
                     if os.path.isdir(os.path.join(results_base_path, d))]
    
    print(f"Found {len(result_folders)} folders")
    
    # Step 1: Find and load the Expected folder
    expected_found = False
    for folder_name in result_folders:
        if "Expected" in folder_name:
            folder_path = os.path.join(results_base_path, folder_name)
            expected_file_path = os.path.join(folder_path, "Stats", "Portfolio", loss_type, "Stats100.parquet")
            
            if os.path.exists(expected_file_path):
                df = pd.read_parquet(expected_file_path)
                df['Expected'] = df['AAL']  # Use AAL for stats instead of Loss
                df['LossType'] = loss_type
                expected_found = True
                print(f"Loaded Expected data from folder: {folder_name}")
                break
    
    if not expected_found:
        print(f"No Expected folder found for {loss_type}")
        return pd.DataFrame()
    
    # Step 2: Process all sample folders
    processed_files = 0
    for folder_name in result_folders:
        if "Expected" not in folder_name:  # Skip the Expected folder
            folder_path = os.path.join(results_base_path, folder_name)
            sample_file_path = os.path.join(folder_path, "Stats", "Portfolio", loss_type, "Stats100.parquet")
            
            if os.path.exists(sample_file_path):
                try:
                    df_temp = pd.read_parquet(sample_file_path)
                    
                    # Extract sample size from folder name
                    sample_size = None
                    match = re.search(r'_S(\d+)_', folder_name)
                    if match:
                        sample_size = int(match.group(1))
                    else:
                        # Fallback: use processed count + 1
                        sample_size = processed_files + 1
                    
                    # Add sample column using AAL
                    column_name = f'S{sample_size}'
                    df[column_name] = df_temp['AAL']  # Use AAL for stats instead of Loss
                    sample_sizes.append(sample_size)
                    processed_files += 1
                    print(f"Added {loss_type} sample from {folder_name}: {column_name}")
                    
                except Exception as e:
                    print(f"Error processing {folder_name}: {str(e)}")
                    continue
    
    sample_sizes.sort()
    print(f"Processed {processed_files} sample folders")
    print(f"Sample sizes: {sample_sizes}")
    
    # Calculate differences vs S32
    for sample_size in sample_sizes:
        s_col = f'S{sample_size}'
        if sample_size != 32 and s_col in df.columns and 'S32' in df.columns:
            df[f'{s_col}VSS32'] = (df[s_col] - df['S32']) / df['S32'] * 100

    # Calculate S32 vs Expected (this is the key comparison)
    if 'S32' in df.columns and 'Expected' in df.columns:
        df['S32vsExpected'] = (df['S32'] - df['Expected']) / df['Expected'] * 100

    # Reorder columns (same as original stats function)
    base_cols = [col for col in df.columns if not col.startswith('S') or col == 'S32vsExpected']

    s_cols = sorted(
        [col for col in df.columns if col.startswith('S') and 'vs' not in col and col[1:].isdigit()],
        key=lambda x: int(x[1:])
    )

    vs_cols = sorted(
        [col for col in df.columns if 'VSS32' in col and col[1:col.find('V')].isdigit()],
        key=lambda x: int(x[1:x.find('V')])
    )

    if 'S32vsExpected' in base_cols:
        base_cols.remove('S32vsExpected')

    new_order = base_cols + s_cols + vs_cols
    if 'S32vsExpected' in df.columns:
        new_order += ['S32vsExpected']
    df = df[new_order]

    return df


In [ ]:
# Plotting Function for Excel Integration
def create_plots_for_excel(df, xlim=None, ylim=None):
    """
    Creates plots for Excel integration with proper legends.
    
    Args:
        df: DataFrame with sample data and comparisons
        xlim: tuple (xmin, xmax) for x-axis limits  
        ylim: tuple (ymin, ymax) for y-axis limits
    
    Returns:
        Dictionary with plot buffers for Excel insertion
    """
    import matplotlib.pyplot as plt
    import io
    
    plots = {}
    
    if df.empty:
        return plots
    
    return_period_col = 'ReturnPeriod'
    vs_cols = [col for col in df.columns if col.endswith('VSS32') and col != 'S32vsExpected']
    
    # Default axis limits if none provided
    default_xlim = (-20, 20)  # % difference default range
    default_ylim = (0, df[return_period_col].max() if return_period_col in df.columns else 1000)
    
    xlim = xlim or default_xlim
    ylim = ylim or default_ylim
    
    print(f"Creating plots with x-axis limits: {xlim}, y-axis limits: {ylim}")
    
    # Plot 1: All sample % differences vs S32
    if vs_cols and return_period_col in df.columns:
        fig1, ax1 = plt.subplots(figsize=(10, 6))
        
        colors = plt.cm.tab10(range(len(vs_cols)))
        for i, col in enumerate(vs_cols):
            if col in df.columns:
                ax1.plot(df[col], df[return_period_col], 
                        label=col, color=colors[i], linewidth=2)
        
        ax1.set_xlabel('% Difference from S32')
        ax1.set_ylabel('Return Period')
        ax1.set_title('All Sample % Differences vs S32')
        ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax1.grid(True, alpha=0.3)
        ax1.set_xlim(xlim)
        ax1.set_ylim(ylim)
        
        buf1 = io.BytesIO()
        fig1.tight_layout()
        fig1.savefig(buf1, format='png', dpi=300, bbox_inches='tight')
        buf1.seek(0)
        plots['all_samples'] = buf1
        plt.close(fig1)
    
    # Plot 2: S24 vs S32 if it exists
    if 'S24VSS32' in df.columns and return_period_col in df.columns:
        fig2, ax2 = plt.subplots(figsize=(8, 6))
        ax2.plot(df['S24VSS32'], df[return_period_col], 
                color='blue', linewidth=2, label='S24 vs S32')
        ax2.set_xlabel('% Difference from S32')
        ax2.set_ylabel('Return Period')
        ax2.set_title('S24 vs S32 % Difference')
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        ax2.set_xlim(xlim)
        ax2.set_ylim(ylim)
        
        buf2 = io.BytesIO()
        fig2.tight_layout()
        fig2.savefig(buf2, format='png', dpi=300, bbox_inches='tight')
        buf2.seek(0)
        plots['s24'] = buf2
        plt.close(fig2)
    
    # Plot 3: S32 vs Expected if it exists
    if 'S32vsExpected' in df.columns and return_period_col in df.columns:
        fig3, ax3 = plt.subplots(figsize=(8, 6))
        ax3.plot(df['S32vsExpected'], df[return_period_col], 
                color='red', linewidth=2, label='S32 vs Expected')
        ax3.set_xlabel('% Difference from Expected')
        ax3.set_ylabel('Return Period')
        ax3.set_title('S32 vs Expected % Difference')
        ax3.grid(True, alpha=0.3)
        ax3.legend()
        ax3.set_xlim(xlim)
        ax3.set_ylim(ylim)
        
        buf3 = io.BytesIO()
        fig3.tight_layout()
        fig3.savefig(buf3, format='png', dpi=300, bbox_inches='tight')
        buf3.seek(0)
        plots['s32_vs_expected'] = buf3
        plt.close(fig3)
    
    return plots


In [ ]:
# Threshold Calculator Function
def calculate_thresholds(df, sheet_name="Analysis"):
    """
    Calculates minimum sample sizes needed to meet 1% and 2% thresholds.
    
    Args:
        df: DataFrame with sample comparison columns
        sheet_name: Name of the sheet being analyzed
    
    Returns:
        Dictionary with threshold results
    """
    print(f"Analyzing thresholds for: {sheet_name}")
    
    if df.empty:
        return {'1%_threshold': None, '2%_threshold': None}
    
    # Filter to return periods 100-1000 if ReturnPeriod column exists
    if 'ReturnPeriod' in df.columns:
        df_filtered = df[(df['ReturnPeriod'] >= 100) & (df['ReturnPeriod'] <= 1000)]
        print(f"Filtered to {len(df_filtered)} rows (return periods 100-1000)")
    else:
        df_filtered = df
        print(f"No ReturnPeriod column found, using all {len(df_filtered)} rows")
    
    if df_filtered.empty:
        print("No data after filtering")
        return {'1%_threshold': None, '2%_threshold': None}
    
    # Find VSS32 columns (sample comparisons)
    vss32_columns = [col for col in df_filtered.columns if 'VSS32' in col and col != 'S32vsExpected']
    
    if not vss32_columns:
        print("No VSS32 comparison columns found")
        return {'1%_threshold': None, '2%_threshold': None}
    
    print(f"Found comparison columns: {vss32_columns}")
    
    min_samples_needed_1 = None
    min_samples_needed_2 = None
    
    # Sort columns by sample number
    sorted_columns = []
    for col in vss32_columns:
        try:
            sample_num = int(col.split('VSS32')[0][1:])  # Extract number from S#VSS32
            sorted_columns.append((sample_num, col))
        except:
            continue
    
    sorted_columns.sort()  # Sort by sample number
    
    for sample_num, col in sorted_columns:
        # Get maximum absolute value for this column
        max_val = df_filtered[col].abs().max()
        
        print(f"S{sample_num}: max absolute value = {max_val:.4f} ({max_val*100:.2f}%)")
        
        # Check thresholds (data is in decimal format: 0.01 = 1%)
        if max_val < 0.01:  # Less than 1%
            if min_samples_needed_1 is None:
                min_samples_needed_1 = sample_num
        
        if max_val < 0.02:  # Less than 2%
            if min_samples_needed_2 is None:
                min_samples_needed_2 = sample_num
    
    result = {
        '1%_threshold': min_samples_needed_1,
        '2%_threshold': min_samples_needed_2
    }
    
    print(f"Results for {sheet_name}:")
    print(f"  Minimum samples for 1% threshold: {min_samples_needed_1}")
    print(f"  Minimum samples for 2% threshold: {min_samples_needed_2}")
    
    return result


In [ ]:
# Integrated Pipeline Function
def run_complete_pipeline(results_base_path, output_excel_path):
    """
    Runs the complete integrated pipeline:
    1. EP Portfolio analysis for GR and GU
    2. Stats Portfolio analysis for GR and GU  
    3. Creates plots with legends
    4. Calculates thresholds
    5. Saves everything to Excel with multiple sheets
    
    Args:
        results_base_path: Path to Results directory containing all folders
        output_excel_path: Path for output Excel file
    """
    import pandas as pd
    import matplotlib.pyplot as plt
    from openpyxl import load_workbook
    from openpyxl.drawing.image import Image
    import tempfile
    import os
    
    print("=" * 80)
    print("STARTING COMPLETE PIPELINE")
    print("=" * 80)
    
    # Step 1: Process all data
    print("\nStep 1: Processing EP Portfolio Data")
    print("-" * 40)
    df_ep_gr = ep_port_aggregating_files(results_base_path, "GR")
    df_ep_gu = ep_port_aggregating_files(results_base_path, "GU")
    
    print("\nStep 2: Processing Stats Portfolio Data")
    print("-" * 40)
    df_stats_gr = stat_port_aggregating_files(results_base_path, "GR")
    df_stats_gu = stat_port_aggregating_files(results_base_path, "GU")
    
    # Step 2: Calculate thresholds
    print("\nStep 3: Calculating Thresholds")
    print("-" * 40)
    threshold_results = {}
    
    if not df_ep_gr.empty:
        threshold_results['EP_GR'] = calculate_thresholds(df_ep_gr, "EP_GR")
    if not df_ep_gu.empty:
        threshold_results['EP_GU'] = calculate_thresholds(df_ep_gu, "EP_GU")
    if not df_stats_gr.empty:
        threshold_results['Stats_GR'] = calculate_thresholds(df_stats_gr, "Stats_GR")
    if not df_stats_gu.empty:
        threshold_results['Stats_GU'] = calculate_thresholds(df_stats_gu, "Stats_GU")
    
    # Step 3: Create plots
    print("\nStep 4: Creating Plots")
    print("-" * 40)
    all_plots = {}
    
    if not df_ep_gr.empty:
        all_plots['EP_GR'] = create_plots_for_excel(df_ep_gr)
        print(f"Created {len(all_plots['EP_GR'])} plots for EP_GR")
    if not df_ep_gu.empty:
        all_plots['EP_GU'] = create_plots_for_excel(df_ep_gu)
        print(f"Created {len(all_plots['EP_GU'])} plots for EP_GU")
    if not df_stats_gr.empty:
        all_plots['Stats_GR'] = create_plots_for_excel(df_stats_gr)
        print(f"Created {len(all_plots['Stats_GR'])} plots for Stats_GR")
    if not df_stats_gu.empty:
        all_plots['Stats_GU'] = create_plots_for_excel(df_stats_gu)
        print(f"Created {len(all_plots['Stats_GU'])} plots for Stats_GU")
    
    # Step 4: Save to Excel
    print("\nStep 5: Saving to Excel")
    print("-" * 40)
    
    # Create threshold summary DataFrame
    threshold_summary = pd.DataFrame.from_dict(threshold_results, orient='index')
    threshold_summary.index.name = 'Analysis_Type'
    threshold_summary = threshold_summary.reset_index()
    
    # Save data to Excel
    with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
        # Save data sheets
        if not df_ep_gr.empty:
            df_ep_gr.to_excel(writer, sheet_name='EP_GR_Data', index=False)
        if not df_ep_gu.empty:
            df_ep_gu.to_excel(writer, sheet_name='EP_GU_Data', index=False)
        if not df_stats_gr.empty:
            df_stats_gr.to_excel(writer, sheet_name='Stats_GR_Data', index=False)
        if not df_stats_gu.empty:
            df_stats_gu.to_excel(writer, sheet_name='Stats_GU_Data', index=False)
        
        # Save threshold results
        threshold_summary.to_excel(writer, sheet_name='Threshold_Results', index=False)
    
    # Step 5: Add plots to Excel
    print("Step 6: Adding Plots to Excel")
    print("-" * 40)
    
    try:
        workbook = load_workbook(output_excel_path)
        
        for analysis_type, plots in all_plots.items():
            if plots:
                # Create a plots sheet for this analysis type
                plot_sheet_name = f'{analysis_type}_Plots'
                if plot_sheet_name in workbook.sheetnames:
                    del workbook[plot_sheet_name]
                plot_sheet = workbook.create_sheet(plot_sheet_name)
                
                row = 1
                for plot_name, plot_buffer in plots.items():
                    # Save plot to temporary file
                    with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmp_file:
                        tmp_file.write(plot_buffer.getvalue())
                        tmp_file_path = tmp_file.name
                    
                    # Add image to Excel
                    img = Image(tmp_file_path)
                    img.width = 600  # Adjust size as needed
                    img.height = 400
                    plot_sheet.add_image(img, f'A{row}')
                    
                    # Add plot title
                    plot_sheet[f'H{row}'] = f'{plot_name.replace("_", " ").title()}'
                    
                    row += 25  # Space between plots
                    
                    # Clean up temporary file
                    os.unlink(tmp_file_path)
        
        workbook.save(output_excel_path)
        print(f"Plots added to Excel successfully")
        
    except Exception as e:
        print(f"Warning: Could not add plots to Excel: {str(e)}")
        print("Data and threshold results were saved successfully")
    
    # Step 6: Summary
    print("\n" + "=" * 80)
    print("PIPELINE COMPLETE")
    print("=" * 80)
    print(f"Output saved to: {output_excel_path}")
    print("\nSummary:")
    print(f"  EP GR data: {'Yes' if not df_ep_gr.empty else 'No data'}")
    print(f"  EP GU data: {'Yes' if not df_ep_gu.empty else 'No data'}")
    print(f"  Stats GR data: {'Yes' if not df_stats_gr.empty else 'No data'}")
    print(f"  Stats GU data: {'Yes' if not df_stats_gu.empty else 'No data'}")
    print(f"  Threshold analysis: {len(threshold_results)} analyses completed")
    print(f"  Total plots created: {sum(len(plots) for plots in all_plots.values())}")
    
    print("\nThreshold Summary:")
    for analysis, results in threshold_results.items():
        print(f"  {analysis}: 1%={results['1%_threshold']}, 2%={results['2%_threshold']}")
    
    return {
        'ep_gr': df_ep_gr,
        'ep_gu': df_ep_gu, 
        'stats_gr': df_stats_gr,
        'stats_gu': df_stats_gu,
        'thresholds': threshold_results,
        'plots': all_plots
    }


In [ ]:
# PIPELINE SUMMARY AND DOCUMENTATION

print("""
================================================================================
COMPLETE PORTFOLIO ANALYSIS PIPELINE
================================================================================

This notebook contains a comprehensive analysis pipeline that:

1. EP PORTFOLIO AGGREGATION (ep_port_aggregating_files)
   - Processes EP portfolio data for GR and GU loss types
   - Finds Expected folder automatically (contains "Expected" in name)
   - Processes all sample folders (S1, S5, S32, etc.)
   - Calculates percentage differences vs S32 and vs Expected
   - Uses Loss column from EP/Portfolio/{loss_type}/EP100.parquet

2. STATS PORTFOLIO AGGREGATION (stat_port_aggregating_files) 
   - Same structure as EP but for Stats data
   - Uses AAL (Average Annual Loss) column
   - Processes Stats/Portfolio/{loss_type}/Stats100.parquet files

3. PLOTTING FUNCTION (create_plots_for_excel)
   - Creates publication-quality plots with legends
   - All sample differences vs S32
   - S24 vs S32 specific plot
   - S32 vs Expected comparison plot
   - High-resolution PNG outputs for Excel embedding

4. THRESHOLD CALCULATOR (calculate_thresholds)
   - Finds minimum sample sizes meeting 1% and 2% thresholds
   - Analyzes return periods 100-1000
   - Based on maximum absolute deviations from S32

5. INTEGRATED PIPELINE (run_complete_pipeline)
   - Runs all analyses automatically
   - Creates Excel file with multiple sheets:
     * Data sheets: EP_GR_Data, EP_GU_Data, Stats_GR_Data, Stats_GU_Data
     * Plot sheets: EP_GR_Plots, EP_GU_Plots, Stats_GR_Plots, Stats_GU_Plots  
     * Threshold_Results: Summary of minimum sample requirements

USAGE:
------
Just update the paths in the last cell and run:

results_base_path = "path/to/your/Results/directory"
output_excel_path = "path/to/output.xlsx"
results = run_complete_pipeline(results_base_path, output_excel_path)

FILE STRUCTURE EXPECTED:
-----------------------
Results/
├── FOLDER_WITH_Expected_IN_NAME/
│   ├── EP/Portfolio/GR/EP100.parquet
│   ├── EP/Portfolio/GU/EP100.parquet
│   ├── Stats/Portfolio/GR/Stats100.parquet
│   └── Stats/Portfolio/GU/Stats100.parquet
├── FOLDER_WITH_S1_IN_NAME/
│   └── (same structure)
├── FOLDER_WITH_S5_IN_NAME/
│   └── (same structure)
└── FOLDER_WITH_S32_IN_NAME/
    └── (same structure)

The pipeline automatically finds all folders and processes them appropriately.
================================================================================
""")

# Quick test function to validate your file structure
def validate_file_structure(results_base_path):
    """Quick validation of expected file structure"""
    if not os.path.exists(results_base_path):
        print(f"ERROR: Results directory not found: {results_base_path}")
        return False
    
    folders = [d for d in os.listdir(results_base_path) 
               if os.path.isdir(os.path.join(results_base_path, d))]
    
    print(f"Found {len(folders)} folders in Results directory")
    
    expected_found = any("Expected" in folder for folder in folders)
    sample_folders = [f for f in folders if "Expected" not in f]
    
    print(f"Expected folder found: {expected_found}")
    print(f"Sample folders: {len(sample_folders)}")
    
    if sample_folders:
        print("Sample folder examples:")
        for folder in sample_folders[:3]:
            print(f"  - {folder}")
        if len(sample_folders) > 3:
            print(f"  ... and {len(sample_folders)-3} more")
    
    return expected_found and len(sample_folders) > 0

# Uncomment to test your file structure:
# validate_file_structure(r"C:\Users\Ashley.Yang\Downloads\DK_RES_GR_GU\Results")


In [ ]:
# COMPLETE INTEGRATED PIPELINE USAGE
# Run this cell to execute the entire pipeline

# Set your paths
results_base_path = r"C:\Users\Ashley.Yang\Downloads\DK_RES_GR_GU\Results"
output_excel_path = r"C:\Users\Ashley.Yang\Downloads\Complete_Portfolio_Analysis.xlsx"

# Run the complete pipeline
print("Starting complete integrated pipeline...")
results = run_complete_pipeline(results_base_path, output_excel_path)

print("\nPipeline completed successfully!")
print(f"Check your Excel file at: {output_excel_path}")
print("\nThe Excel file contains:")
print("- Data sheets: EP_GR_Data, EP_GU_Data, Stats_GR_Data, Stats_GU_Data")
print("- Plot sheets: EP_GR_Plots, EP_GU_Plots, Stats_GR_Plots, Stats_GU_Plots")
print("- Threshold_Results sheet with summary of minimum sample requirements")


In [ ]:
df_ep_gr = stat_port_aggregating_files(r"", "GR")

In [ ]:
import matplotlib.pyplot as plt
import io
#Generate plots where the x-axis is the return period and the y-axis is the 
def create_ep_port_plots(df, 
                 xlim=None,  # tuple (xmin, xmax) for x-axis limits
                 ylim=None   # tuple (ymin, ymax) for y-axis limits
                ):
    plots = {}

    return_period_col = 'ReturnPeriod'
    vs_cols = [col for col in df.columns if col.endswith('VSS32') and col != 'S32vsExpected']

    # Default axis limits if none provided
    default_xlim = (-20, 20)  # % difference default range
    default_ylim = (0, df[return_period_col].max())  # full y range by default

    xlim = xlim or default_xlim
    ylim = ylim or default_ylim

    print(f"Using x-axis limits: {xlim}")
    print(f"Using y-axis limits: {ylim}")

    # Plot 1: All % diff lines (e.g., S1VSS32, S5VSS32...)
    fig1, ax1 = plt.subplots(figsize=(7, 5))
    for col in vs_cols:
        if col in df.columns:
            ax1.plot(df[col], df[return_period_col], label=col)
    ax1.set_xlabel('% Difference from S32')
    ax1.set_ylabel('Return Period')
    ax1.set_title('All Sample % Diffs vs S32')
    ax1.legend()
    ax1.grid(True)

    ax1.set_xlim(xlim)
    ax1.set_ylim(ylim)

    buf1 = io.BytesIO()
    fig1.tight_layout()
    fig1.savefig(buf1, format='png')
    buf1.seek(0)
    plots['all_samples'] = buf1
    plt.close(fig1)

    # Plot 2: Just S24VSS32 if it exists
    fig2, ax2 = plt.subplots(figsize=(7, 5))
    if 'S24VSS32' in df.columns:
        ax2.plot(df['S24VSS32'], df[return_period_col], color='blue', label='S24VSS32')
        ax2.set_xlabel('% Difference from S32')
        ax2.set_ylabel('Return Period')
        ax2.set_title('S24 vs S32 % Difference')
        ax2.grid(True)
        ax2.legend()

        ax2.set_xlim(xlim)
        ax2.set_ylim(ylim)

        buf2 = io.BytesIO()
        fig2.tight_layout()
        fig2.savefig(buf2, format='png')
        buf2.seek(0)
        plots['s24'] = buf2
    plt.close(fig2)

    return plots


In [ ]:
#find which sample # is within the threshold of +/- 1% and 2% for ALL sheets
import pandas as pd

# Read all sheets from the Excel file
excel_file = r'C:\Users\Ashley.Yang\Downloads\DK_RES_GR_GU\DK_RES_GR_GU_Aggregation.xlsx'
all_sheets = pd.read_excel(excel_file, sheet_name=None, header=0, index_col=0)

#look for the columns: 
columns = ['S1VSS32', 'S2VSS32', 'S4VSS32', 'S5VSS32', 'S6VSS32','S10VSS32', 'S12VSS32', 'S16VSS32', 'S24VSS32']

# Store results for all sheets
results = {}

# Loop through each sheet
for sheet_name, df in all_sheets.items():
    if df.columns.contains("EPType"):
        continue
    else:
        break
    print(f"ANALYZING SHEET: {sheet_name}")
    #only want the rows where return period is between 100 and 1000
    if 'ReturnPeriod' in df.columns:
        df = df[(df['ReturnPeriod'] >= 100) & (df['ReturnPeriod'] <= 1000)]
        print(f"Filtered to {len(df)} rows (return periods 100-1000)")
    # Skip if the required columns don't exist
    missing_cols = [col for col in columns if col not in df.columns]
    if missing_cols:
        print(f"Missing columns in {sheet_name}: {missing_cols}")
        continue
    
    # Only keep when the return period is between 100 and 1000
    if 'ReturnPeriod' in df.columns:
        df_filtered = df[(df['ReturnPeriod'] >= 100) & (df['ReturnPeriod'] <= 1000)]
        print(f"Filtered to {len(df_filtered)} rows (return periods 100-1000)")
    else:
        df_filtered = df
        print(f"No ReturnPeriod column found, using all {len(df_filtered)} rows")
    
    # Initialize for this sheet
    min_samples_needed_1 = None
    min_samples_needed_2 = None

    for col in columns:
        # Extract the sample number from column name 
        # Handle both 'S12VS32' and 'S#VSS32' formats
        if 'VS32' in col and 'VSS32' not in col:  # S12VS32 case
            sample_num = int(col.split('VS32')[0][1:])  # Remove 'S' prefix and 'VS32' suffix
        else:  # S#VSS32 case
            sample_num = int(col.split('VSS32')[0][1:])  # Remove 'S' prefix and 'VSS32' suffix
        
        # Debug: Show the actual max value for each column
        max_val = max(abs(df_filtered[col]))
        max_val_percent = max_val  # Convert to percentage for display
        print(f"{col}: max absolute value = {max_val:.6f} (={max_val_percent:.2f}%)")
        
        # Check if this sample size meets the 1% threshold
        # Data is in decimal format (0.01 = 1%)
        if max_val < 0.01:  # Less than 1% (0.01 in decimal)
            if min_samples_needed_1 is None:  # Only set if we haven't found one yet
                min_samples_needed_1 = sample_num
        
        # Check if this sample size meets the 2% threshold  
        if max_val < 0.02:  # Less than 2% (0.02 in decimal)
            if min_samples_needed_2 is None:  # Only set if we haven't found one yet
                min_samples_needed_2 = sample_num

    # Store results for this sheet
    results[sheet_name] = {
        '1%_threshold': min_samples_needed_1,
        '2%_threshold': min_samples_needed_2
    }
    
    print(f"\nRESULTS FOR {sheet_name}:")
    print(f"Minimum samples needed for ±1% threshold: {min_samples_needed_1}")
    print(f"Minimum samples needed for ±2% threshold: {min_samples_needed_2}")

# Print summary of all results
print(f"\n{'='*60}")
print("SUMMARY FOR ALL SHEETS")
print(f"{'='*60}")
print(f"{'Sheet Name':<20} {'±1% Threshold':<15} {'±2% Threshold':<15}")
print("-" * 60)
for sheet, result in results.items():
    print(f"{sheet:<20} {str(result['1%_threshold']):<15} {str(result['2%_threshold']):<15}")


In [ ]:

    # Generate plots again (with legend extraction)
    def create_ep_port_plots_with_legend(df):
        plots = {}
        legends = {}

        return_period_col = 'ReturnPeriod'
        vs_cols = [col for col in df.columns if col.endswith('VSS32') and col != 'S32vsExpected']
        cmap = get_cmap('tab20')  # Use matplotlib tab20 color map

        fig1, ax1 = plt.subplots(figsize=(7, 5))
        color_map = {}

        for i, col in enumerate(vs_cols):
            color = cmap(i % 20)
            hex_color = to_hex(color)
            color_map[col] = hex_color
            ax1.plot(df[return_period_col],df[col], label=col, color=color)

        ax1.set_xlabel('Return Period')
        ax1.set_ylabel('% Difference from S32')
        ax1.set_title('All Sample % Diffs vs S32')
        ax1.grid(True)
        ax1.set_xlim((0, df[return_period_col].max()))
        ax1.set_ylim((-10,10))
        ax1.legend()

        buf1 = io.BytesIO()
        fig1.tight_layout()
        fig1.savefig(buf1, format='png')
        buf1.seek(0)
        plots['all_samples'] = buf1
        legends['all_samples'] = color_map
        plt.close(fig1)

        # Plot S24VSS32 if available
        if 'S24VSS32' in df.columns:
            fig2, ax2 = plt.subplots(figsize=(7, 5))
            ax2.plot( df[return_period_col], df['S24VSS32'], color='blue', label='S24VSS32')
            ax2.set_ylabel('% Difference from S32')
            ax2.set_xlabel('Return Period')
            ax2.set_title('S24 vs S32 % Difference')
            ax2.grid(True)
            ax2.legend()
            ax2.set_ylim((-10, 10))
            ax2.set_xlim((0, df[return_period_col].max()))

            buf2 = io.BytesIO()
            fig2.tight_layout()
            fig2.savefig(buf2, format='png')
            buf2.seek(0)
            plots['s24'] = buf2
            legends['s24'] = {'S24VSS32': '#0000FF'}  # blue
            plt.close(fig2)

        return plots, legends

    def insert_plots_with_legend(worksheet, plots, legends):
        row = 0
        for key, buf in plots.items():
            # Insert the plot image
            image_data = buf.read()
            worksheet.insert_image(row, 0, f"{key}.png", {'image_data': io.BytesIO(image_data)})

            # Insert the legend table beside it
            legend_map = legends.get(key, {})
            worksheet.write(row, 8, "Legend:")
            worksheet.write(row + 1, 8, "Line Name")
            worksheet.write(row + 1, 9, "Color (Hex)")

            for i, (line_name, hex_color) in enumerate(legend_map.items()):
                worksheet.write(row + 2 + i, 8, line_name)
                worksheet.write(row + 2 + i, 9, hex_color)

            row += 35  # Space between plots

    # EP_GR
    plots_gr, legends_gr = create_ep_port_plots_with_legend(EP_GR)
    worksheet_gr = workbook.add_worksheet("EP_GR_Plots")
    writer.sheets["EP_GR_Plots"] = worksheet_gr
    insert_plots_with_legend(worksheet_gr, plots_gr, legends_gr)

    # EP_GU
    plots_gu, legends_gu = create_ep_port_plots_with_legend(EP_GU)
    worksheet_gu = workbook.add_worksheet("EP_GU_Plots")
    writer.sheets["EP_GU_Plots"] = worksheet_gu
    insert_plots_with_legend(worksheet_gu, plots_gu, legends_gu)
